In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import os

In [2]:
# Desactivo warning molesto
pd.options.mode.chained_assignment = None  # default='warn'
os.getcwd()

'/home/cesar/universidad/7506/tp/2018_2C/tp2/tp2_lituarte/Datos-TP2'

In [3]:
os.chdir('/home/cesar/universidad/7506/tp/2018_2C/tp2/fiuba-trocafone-tp2-final-set/')

In [4]:
eventos = pd.read_csv('events_up_to_01062018.csv', low_memory=False)

In [5]:
labels = pd.read_csv('labels_training_set.csv')

In [6]:
eventos.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
labels.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [8]:
eventos["timestamp"] = pd.to_datetime(eventos["timestamp"])

El csv de labels tiene las personas y un booleano indicando si compraron. Por lo tanto mi objetivo es transformar el csv de eventos en uno que tenga información para cada persona.

In [9]:
# Como estoy evaluando a los usuarios no me interesa los eventos que no tengan una persona asociada
eventos.loc[eventos['person'].isnull() == True,:]

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version


In [10]:
eventos["busquedas_exactas"] = (eventos["search_term"].notnull() & eventos["search_term"].isin(eventos['model'].unique()))

In [11]:
eventos["terminos_buscados"] = eventos['search_term'].replace(np.nan, '', regex=True).str.split().apply(len)

In [12]:
eventos['marca'] = eventos['model'].str.split(' ', n = 1, expand = True)[0]

In [13]:
eventos['Samsung'] = (eventos['marca'] == 'Samsung')
eventos['iPhone'] = (eventos['marca'] == 'iPhone')
eventos['LG'] = (eventos['marca'] == 'LG')
eventos['Motorola'] = (eventos['marca'] == 'Motorola')
eventos['iPad'] = (eventos['marca'] == 'iPad')
eventos['Sony'] = (eventos['marca'] == 'Sony')
eventos['Lenovo'] = (eventos['marca'] == 'Lenovo')
eventos['Quantum'] = (eventos['marca'] == 'Quantum')
eventos['Asus'] = (eventos['marca'] == 'Asus')
eventos['Xiaomi'] = (eventos['marca'] == 'Xiaomi')
eventos['Outros'] = (eventos['marca'] == 'Outros')

In [14]:
eventos['Bom'] = (eventos['condition'] == 'Bom')
eventos['Muito_Bom'] = (eventos['condition'] == 'Muito Bom')
eventos['Excelente'] = (eventos['condition'] == 'Excelente')

Se me ocurre que podría extraer la columa "event" y hacer una columna por cada evento con un booleano, esto me ayudaria a contar la cantidad de veces que se registraron los eventos para cada usuario.

In [15]:
eventos['viewed_product'] = (eventos['event'] == 'viewed product')
eventos['brand_listing'] = (eventos['event'] == 'brand listing')
eventos['visited_site'] = (eventos['event'] == 'visited site')
eventos['ad_campaign_hit'] = (eventos['event'] == 'ad campaign hit')
eventos['generic_listing'] = (eventos['event'] == 'generic listing')
eventos['searched_products'] = (eventos['event'] == 'searched products')
eventos['search_engine_hit'] = (eventos['event'] == 'search engine hit')
eventos['checkout'] = (eventos['event'] == 'checkout')
eventos['staticpage'] = (eventos['event'] == 'staticpage')
eventos['conversion'] = (eventos['event'] == 'conversion')
eventos['lead'] = (eventos['event'] == 'lead')
eventos['total_eventos'] = 1;

In [16]:
eventos.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,brand_listing,visited_site,ad_campaign_hit,generic_listing,searched_products,search_engine_hit,checkout,conversion,lead,total_eventos
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,False,False,False,False,False,False,False,False,False,1
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,False,False,False,False,False,False,False,False,False,1
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,False,False,False,False,False,False,False,False,False,1
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,False,False,False,False,False,False,False,False,False,1
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,False,False,False,False,False,False,False,False,False,1


La columna "timestamp" me parece muy importante pero como voy a tener que agrupar todos los eventos de una persona a una sola fila necesito hacer algo con ella. Se me ocurre que puedo agregar columnas para determinar si cada evento tuvo lugar por ejemplo una columa por mes. Esta creación de columnas va a estar muy ligada al hecho que los datos que tenemos van desde principio de este año hasta junio.

In [17]:
eventos["mes"] = eventos["timestamp"].dt.month
eventos["dia_del_anio"] = eventos["timestamp"].dt.dayofyear
eventos["semana"] = eventos["timestamp"].dt.week

In [18]:
# Obtengo la ultima fecha disponible en el csv para tener un punto de partida
ultima_fecha = eventos["timestamp"].max()
ultimo_dia = ultima_fecha.dayofyear
ultimo_mes = ultima_fecha.month
ultima_semana = ultima_fecha.week

In [19]:
# Agrego columna para obtener los dias desde el evento
eventos["dias_ultimo_evento"] = (ultimo_dia - eventos["dia_del_anio"])

In [20]:
# Agrego columna para saber hace cuanto compro en la pagina
eventos["dias_ultima_compra"] = ultimo_dia
eventos.loc[eventos["conversion"] == True, "dias_ultima_compra"] = ultimo_dia - eventos["dia_del_anio"]

In [21]:
# Agrego columnas para los m meses más cercanos
for m in range (0, ultimo_mes):
    eventos["mes_pasado_" + str(m)] = (eventos["mes"] == (ultimo_mes - m))
    eventos['viewed_product_mes_pasado_' + str(m)] = ((eventos['event'] == 'viewed product') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['brand_listing_mes_pasado_' + str(m)] = ((eventos['event'] == 'brand listing') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['visited_site_mes_pasado_' + str(m)] = ((eventos['event'] == 'visited site') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['ad_campaign_hit_mes_pasado_' + str(m)] = ((eventos['event'] == 'ad campaign hit') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['generic_listing_mes_pasado_' + str(m)] = ((eventos['event'] == 'generic listing') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['searched_products_mes_pasado_' + str(m)] = ((eventos['event'] == 'searched products') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['search_engine_hit_mes_pasado_' + str(m)] = ((eventos['event'] == 'search engine hit') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['checkout_mes_pasado_' + str(m)] = ((eventos['event'] == 'checkout') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['staticpage_mes_pasado_' + str(m)] = ((eventos['event'] == 'staticpage') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['conversion_mes_pasado_' + str(m)] = ((eventos['event'] == 'conversion') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['lead_mes_pasado_' + str(m)] = ((eventos['event'] == 'lead') & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Bom_mes_pasado_' + str(m)] = ((eventos['Bom'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Muito_Bom_mes_pasado_' + str(m)] = ((eventos['Muito_Bom'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Excelente_mes_pasado_' + str(m)] = ((eventos['Excelente'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Samsung_mes_pasado_' + str(m)] = ((eventos['Samsung'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['iPhone_mes_pasado_' + str(m)] = ((eventos['iPhone'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['LG_mes_pasado_' + str(m)] = ((eventos['LG'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Motorola_mes_pasado_' + str(m)] = ((eventos['Motorola'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['iPad_mes_pasado_' + str(m)] = ((eventos['iPad'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Sony_mes_pasado_' + str(m)] = ((eventos['Sony'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Lenovo_mes_pasado_' + str(m)] = ((eventos['Lenovo'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Quantum_mes_pasado_' + str(m)] = ((eventos['Quantum'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Asus_mes_pasado_' + str(m)] = ((eventos['Asus'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Xiaomi_mes_pasado_' + str(m)] = ((eventos['Xiaomi'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['Outros_mes_pasado_' + str(m)] = ((eventos['Outros'] == True) & (eventos["mes_pasado_" + str(m)] == True))
    eventos['busquedas_exactas_mes_pasado_' + str(m)] = ((eventos['busquedas_exactas'] == True) & (eventos["mes_pasado_" + str(m)] == True))

In [22]:
eventos.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,LG_mes_pasado_4,Motorola_mes_pasado_4,iPad_mes_pasado_4,Sony_mes_pasado_4,Lenovo_mes_pasado_4,Quantum_mes_pasado_4,Asus_mes_pasado_4,Xiaomi_mes_pasado_4,Outros_mes_pasado_4,busquedas_exactas_mes_pasado_4
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,False,False,False,False,False,False,False,False,False,False
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,False,False,False,False,False,False,False,False,False,False
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,False,False,False,False,False,False,False,False,False,False
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,False,False,False,False,False,False,False,False,False,False
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,False,False,False,False,False,False,False,False,False,False


In [23]:
# Agrego columnas para las s semanas más cercanas
for s in range (0, 5):
    eventos["semana_pasada_" + str(s)] = (eventos["semana"] == (ultima_semana - s))

In [24]:
eventos.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,Quantum_mes_pasado_4,Asus_mes_pasado_4,Xiaomi_mes_pasado_4,Outros_mes_pasado_4,busquedas_exactas_mes_pasado_4,semana_pasada_0,semana_pasada_1,semana_pasada_2,semana_pasada_3,semana_pasada_4
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,False,False,False,False,False,False,False,True,False,False
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,False,False,False,False,False,False,False,True,False,False
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,False,False,False,False,False,False,False,True,False,False
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,False,False,False,False,False,False,False,True,False,False
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,False,False,False,False,False,False,False,True,False,False


eventos = eventos.loc[:,['person', 'viewed_product', 'brand_listing', 'visited_site',
       'ad_campaign_hit', 'generic_listing', 'searched_products',
       'search_engine_hit', 'checkout', 'conversion', 'lead',
'mes_pasado_0', 'mes_pasado_1', 'mes_pasado_2', 'mes_pasado_3',
     'semana_pasada_0', 'semana_pasada_1', 'semana_pasada_2',
     'semana_pasada_3', 'semana_pasada_4']]

In [25]:
eventos.columns

Index([u'timestamp', u'event', u'person', u'url', u'sku', u'model',
       u'condition', u'storage', u'color', u'skus',
       ...
       u'Quantum_mes_pasado_4', u'Asus_mes_pasado_4', u'Xiaomi_mes_pasado_4',
       u'Outros_mes_pasado_4', u'busquedas_exactas_mes_pasado_4',
       u'semana_pasada_0', u'semana_pasada_1', u'semana_pasada_2',
       u'semana_pasada_3', u'semana_pasada_4'],
      dtype='object', length=196)

In [26]:
usuarios = eventos.groupby('person', as_index=False).agg({
                                         'conversion' : 'sum',
                                         'viewed_product_mes_pasado_0':'sum', 
                                         'brand_listing_mes_pasado_0':'sum',
                                         'visited_site_mes_pasado_0':'sum', 
                                         'ad_campaign_hit_mes_pasado_0': 'sum',
                                         'generic_listing_mes_pasado_0' : 'sum',
                                         'searched_products_mes_pasado_0' : 'sum',
                                         'search_engine_hit_mes_pasado_0' : 'sum',
                                         'checkout_mes_pasado_0' : 'sum',
                                         'conversion_mes_pasado_0' : 'sum',
                                         'mes_pasado_0' : 'sum',
                                         'mes_pasado_1' : 'sum',
                                         'semana_pasada_0' : 'sum',
                                         'semana_pasada_1' : 'sum',
                                         'semana_pasada_2' : 'sum',
                                         'semana_pasada_3' : 'sum',
                                         'semana_pasada_4' : 'sum',
                                         'dias_ultimo_evento' : 'min',
                                         'dias_ultima_compra' : 'min',
                                         'Bom_mes_pasado_0' : 'sum',
                                         'Muito_Bom_mes_pasado_0' : 'sum',
                                         'Excelente_mes_pasado_0' : 'sum',
                                         'Samsung_mes_pasado_0' : 'sum',
                                         'iPhone_mes_pasado_0' : 'sum'})

In [27]:
# Incluyo solo los eventos de los usuarios que tengo labels
usuarios_con_labels = usuarios.loc[usuarios['person'].isin(labels['person'])]

In [28]:
usuarios_con_labels = usuarios_con_labels.set_index('person')

In [29]:
labels = labels.set_index('person')

In [30]:
usuarios_con_labels = usuarios_con_labels.join(labels, lsuffix='person', rsuffix='person')

In [31]:
usuarios_con_labels.head()

,brand_listing_mes_pasado_0,semana_pasada_4,search_engine_hit_mes_pasado_0,ad_campaign_hit_mes_pasado_0,mes_pasado_0,Bom_mes_pasado_0,conversion,semana_pasada_2,semana_pasada_3,semana_pasada_0,...,Excelente_mes_pasado_0,visited_site_mes_pasado_0,iPhone_mes_pasado_0,Muito_Bom_mes_pasado_0,searched_products_mes_pasado_0,generic_listing_mes_pasado_0,Samsung_mes_pasado_0,dias_ultimo_evento,viewed_product_mes_pasado_0,label
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,0.0,0.0,0.0,0.0,6.0,2.0,0.0,6.0,0.0,0.0,...,0.0,2.0,1.0,1.0,0.0,1.0,1.0,14,0.0,0
000c79fe,0.0,0.0,1.0,1.0,17.0,4.0,0.0,0.0,0.0,17.0,...,0.0,1.0,4.0,0.0,9.0,1.0,0.0,2,3.0,0
001802e4,0.0,0.0,0.0,5.0,19.0,3.0,0.0,0.0,0.0,19.0,...,2.0,1.0,5.0,0.0,4.0,4.0,0.0,0,4.0,0
0019e639,113.0,108.0,7.0,16.0,290.0,56.0,2.0,181.0,0.0,0.0,...,20.0,12.0,0.0,52.0,1.0,13.0,21.0,9,123.0,0
001b0bf9,1.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,1.0,5.0,9,2.0,0


In [32]:
usuarios_con_labels.sort_values('viewed_product_mes_pasado_0', ascending = False).head(100)

,brand_listing_mes_pasado_0,semana_pasada_4,search_engine_hit_mes_pasado_0,ad_campaign_hit_mes_pasado_0,mes_pasado_0,Bom_mes_pasado_0,conversion,semana_pasada_2,semana_pasada_3,semana_pasada_0,...,Excelente_mes_pasado_0,visited_site_mes_pasado_0,iPhone_mes_pasado_0,Muito_Bom_mes_pasado_0,searched_products_mes_pasado_0,generic_listing_mes_pasado_0,Samsung_mes_pasado_0,dias_ultimo_evento,viewed_product_mes_pasado_0,label
person,,,,,,,,,,,,,,,,,,,,,
6abd2bf1,5.0,210.0,1.0,0.0,1334.0,496.0,1.0,272.0,341.0,84.0,...,257.0,70.0,983.0,363.0,1.0,90.0,46.0,0,1163.0,0
52a4f0b6,10.0,0.0,3.0,5.0,1101.0,391.0,0.0,918.0,0.0,175.0,...,301.0,10.0,100.0,364.0,2.0,6.0,293.0,3,1062.0,0
18489dd5,43.0,0.0,5.0,24.0,1563.0,308.0,0.0,369.0,422.0,369.0,...,243.0,48.0,497.0,430.0,314.0,62.0,494.0,0,1050.0,0
595b9b50,12.0,272.0,7.0,29.0,1179.0,488.0,0.0,8.0,0.0,292.0,...,154.0,27.0,30.0,401.0,3.0,54.0,840.0,2,1043.0,0
58cb5f05,14.0,0.0,11.0,6.0,1059.0,266.0,0.0,123.0,305.0,326.0,...,323.0,26.0,78.0,376.0,25.0,11.0,777.0,1,964.0,0
3787229d,18.0,58.0,35.0,20.0,1113.0,253.0,1.0,434.0,105.0,161.0,...,394.0,55.0,197.0,294.0,6.0,23.0,478.0,0,952.0,0
c8e25442,5.0,348.0,6.0,26.0,1042.0,207.0,0.0,70.0,161.0,34.0,...,572.0,52.0,497.0,122.0,0.0,41.0,389.0,0,911.0,0
f2408126,16.0,0.0,4.0,6.0,941.0,335.0,0.0,0.0,0.0,807.0,...,255.0,16.0,352.0,289.0,7.0,10.0,492.0,0,879.0,1
0d43b999,25.0,99.0,5.0,2.0,1095.0,253.0,1.0,164.0,246.0,155.0,...,339.0,100.0,80.0,266.0,7.0,88.0,669.0,1,862.0,0


In [33]:
usuarios_con_labels = usuarios_con_labels.loc[usuarios_con_labels['viewed_product_mes_pasado_0'] < 500, :]

In [34]:
X = usuarios_con_labels.iloc[:,:-1]

In [35]:
X.head()

,brand_listing_mes_pasado_0,semana_pasada_4,search_engine_hit_mes_pasado_0,ad_campaign_hit_mes_pasado_0,mes_pasado_0,Bom_mes_pasado_0,conversion,semana_pasada_2,semana_pasada_3,semana_pasada_0,...,dias_ultima_compra,Excelente_mes_pasado_0,visited_site_mes_pasado_0,iPhone_mes_pasado_0,Muito_Bom_mes_pasado_0,searched_products_mes_pasado_0,generic_listing_mes_pasado_0,Samsung_mes_pasado_0,dias_ultimo_evento,viewed_product_mes_pasado_0
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,0.0,0.0,0.0,0.0,6.0,2.0,0.0,6.0,0.0,0.0,...,151,0.0,2.0,1.0,1.0,0.0,1.0,1.0,14,0.0
000c79fe,0.0,0.0,1.0,1.0,17.0,4.0,0.0,0.0,0.0,17.0,...,151,0.0,1.0,4.0,0.0,9.0,1.0,0.0,2,3.0
001802e4,0.0,0.0,0.0,5.0,19.0,3.0,0.0,0.0,0.0,19.0,...,151,2.0,1.0,5.0,0.0,4.0,4.0,0.0,0,4.0
0019e639,113.0,108.0,7.0,16.0,290.0,56.0,2.0,181.0,0.0,0.0,...,9,20.0,12.0,0.0,52.0,1.0,13.0,21.0,9,123.0
001b0bf9,1.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,0.0,0.0,...,9,5.0,0.0,0.0,0.0,0.0,1.0,5.0,9,2.0


In [36]:
Y = usuarios_con_labels.iloc[:,-1]

In [37]:
Y.head()

person
0008ed71    0
000c79fe    0
001802e4    0
0019e639    0
001b0bf9    0
Name: label, dtype: int64

In [38]:
# FIN DE LA PREPARACION DE LOS DATAFRAMES

In [39]:
#evaluacion de modelos
from sklearn.model_selection import cross_validate
def rmse(y_real, y_pred):
    return np.sqrt(mean_squared_error(y_real, y_pred))

def rmse_cv(estimator, X, y):
    y_pred = estimator.predict(X)
    return rmse(y, y_pred)

def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring="roc_auc", n_jobs=-1, cv=10, return_train_score=True)
    return resultados_estimador

resultados = {}

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].min()
    return resultados_df

In [40]:
#Creo muestra de entrenamiento y de test
#predeccion train = X_train
#prediccion test  = X_test
#target     train = y_train
#target     test  = y_test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2,random_state=123)


#Import de Random Forest Classifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
#n_estitamator = numero de arboles que vamos a construir
rf_regressor = RandomForestRegressor(n_estimators=100)
#modelo sobre los datos de entrenamiento
rf_regressor = rf_regressor.fit(X_train,y_train)

#Predecimos para los valores del grupo Test
predictions = rf_regressor.predict(X_test)


In [41]:
## MEJORA DE HIPERPARAMETROS
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint

param_dist_random = {
    "max_depth": [5, None],
    "max_features": sp_randint(1, 11),
    "min_samples_split": sp_randint(2, 11),
    "min_samples_leaf": sp_randint(1, 11),
    "bootstrap": [True, False],
    "n_estimators": np.linspace(10,1000,10).astype(int)
   
}
parametros_busqueda_rf = {
    "criterion": ["gini", "entropy"],
    "n_estimators": np.linspace(10,1000,10).astype(int),
    "class_weight": [None, "balanced"]
}

In [42]:
rfr_model = RandomForestRegressor()
#clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
#                   cv=StratifiedKFold(Y, n_folds=5, shuffle=True), 
#                   scoring='roc_auc',
#                   verbose=2, refit=True)

clf = RandomizedSearchCV(estimator=rfr_model, 
                    param_distributions=param_dist_random,
                   scoring="roc_auc", n_jobs=-1, n_iter=10, random_state=7)

In [43]:
clf.fit(X_train,y_train)

/home/cesar/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe0f9bf9d90>, 'n_estimators': array([  10,  120,  230,  340,  450,  560,  670,  780,  890, 1000]), 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe0f9bf9b10>, 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe0f9bf99d0>, 'max_depth': [5, None]},
          pre_dispatch=

In [44]:
print(clf.best_score_)
print(clf.best_estimator_)

0.851746720071
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features=9, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=8,
           min_samples_split=8, min_weight_fraction_leaf=0.0,
           n_estimators=450, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)


In [45]:
resultados["rfr_randomsearch"] = evaluar_modelo(clf.best_estimator_, X, Y)
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
rfr_randomsearch,9.388139,0.079511,0.856719,0.879324,1.0,1.0,1.0,1.0


In [46]:
#best_parameters, score, _ = max(clf.cv_results_, key=lambda x: x[1])
#print('Raw AUC score:', score)
#for param_name in sorted(best_parameters.keys()):
#    print("%s: %r" % (param_name, best_parameters[param_name]))

In [47]:
# Ahora tengo que predecir los de prueba de Kaggle
ids_usuarios_kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [48]:
x_test_kaggle = usuarios.loc[usuarios['person'].isin(ids_usuarios_kaggle['person']), :]

In [49]:
x_test_kaggle.head()

,person,brand_listing_mes_pasado_0,semana_pasada_4,search_engine_hit_mes_pasado_0,ad_campaign_hit_mes_pasado_0,mes_pasado_0,Bom_mes_pasado_0,conversion,semana_pasada_2,semana_pasada_3,...,dias_ultima_compra,Excelente_mes_pasado_0,visited_site_mes_pasado_0,iPhone_mes_pasado_0,Muito_Bom_mes_pasado_0,searched_products_mes_pasado_0,generic_listing_mes_pasado_0,Samsung_mes_pasado_0,dias_ultimo_evento,viewed_product_mes_pasado_0
1,00091926,25.0,33.0,0.0,15.0,448.0,102.0,0.0,167.0,132.0,...,151,108.0,34.0,253.0,163.0,0.0,0.0,61.0,0,372.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66,0.0
3,000ba417,24.0,0.0,1.0,1.0,206.0,115.0,1.0,68.0,0.0,...,5,11.0,6.0,8.0,34.0,0.0,14.0,110.0,5,153.0
5,000e4d9e,17.0,0.0,5.0,19.0,411.0,124.0,0.0,195.0,0.0,...,151,53.0,13.0,21.0,163.0,0.0,17.0,301.0,1,339.0
6,000e619d,11.0,0.0,3.0,6.0,68.0,14.0,0.0,68.0,0.0,...,151,7.0,5.0,7.0,8.0,6.0,8.0,12.0,13,28.0


In [50]:
x_test_kaggle = x_test_kaggle.set_index('person')

In [65]:
#aplico el mejor.
rf_regressor = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features=9, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=8,
           min_samples_split=8, min_weight_fraction_leaf=0.0,
           n_estimators=450, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)


#modelo sobre los datos de entrenamiento
rf_regressor = rf_regressor.fit(X_train,y_train)

In [66]:
# Predecimos
preds_kaggle = rf_regressor.predict(x_test_kaggle)

In [67]:
preds_kaggle

array([ 0.0586347 ,  0.14540147,  0.04409452, ...,  0.13066312,
        0.05346952,  0.0079473 ])

In [68]:
pd.DataFrame({'person': x_test_kaggle.index, 'label': preds_kaggle}).to_csv('/home/cesar/universidad/7506/tp/2018_2C/tp2/fiuba-trocafone-tp2-final-set/test_cesar_rfrv2.csv', index=False, columns=['person', 'label'])

In [69]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot


In [ ]:
# calculate AUC
auc = roc_auc_score(y_test, predictions)
print('AUC: %.6f' % auc)

AUC: 0.821419


In [ ]:
# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, predictions)
# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
# show the plot
pyplot.show()